In [2]:
import pandas as pd
import numpy as np
from unidecode import unidecode

from embrapa_api.preprocessing.preprocessors import BasePreprocessor

# Producao

In [3]:
rw_producao = (
    pd.read_csv(
        'http://vitibrasil.cnpuv.embrapa.br/download/Producao.csv',
        sep=';'
    )
)

rf_producao = rw_producao.copy()

In [4]:
rf_producao = (
    rf_producao
    .melt(
        id_vars=["id", "produto"],
        var_name="ano",
        value_name="producao_L"
    )
    .rename(columns={
        "id": "ID_PRODUTO",
        "produto": "NM_PRODUTO",
        "ano": "DT_ANO",
        "producao_L": "VR_PRODUCAO_L"
    })
    .astype({
        "ID_PRODUTO": int,
        "NM_PRODUTO": str,
        "DT_ANO": int,
        "VR_PRODUCAO_L": float
    })
    .sort_values(by=["ID_PRODUTO", "DT_ANO"])
)
rf_producao["NM_PRODUTO"] = rf_producao["NM_PRODUTO"].apply(unidecode).str.title()

In [5]:
rf_producao.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2703 entries, 0 to 2702
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_PRODUTO     2703 non-null   int64  
 1   NM_PRODUTO     2703 non-null   object 
 2   DT_ANO         2703 non-null   int64  
 3   VR_PRODUCAO_L  2703 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 105.6+ KB


In [7]:
assert (rf_producao.groupby(["ID_PRODUTO"])["NM_PRODUTO"].nunique() == 1).all, "garantir que o nome do produto é único por ID_PRODUTO"

In [6]:
rf_producao.head()

,ID_PRODUTO,NM_PRODUTO,DT_ANO,VR_PRODUCAO_L
0,1,Vinho De Mesa,1970,217208604.0
51,1,Vinho De Mesa,1971,154264651.0
102,1,Vinho De Mesa,1972,146953297.0
153,1,Vinho De Mesa,1973,116710345.0
204,1,Vinho De Mesa,1974,193875345.0


# Processamento

In [94]:
processing_dict = {
    "Viniferas": {
        "url": 'http://vitibrasil.cnpuv.embrapa.br/download/ProcessaViniferas.csv'
},
    "Americanas": {
        "url": 'http://vitibrasil.cnpuv.embrapa.br/download/ProcessaAmericanas.csv'
    },
    "Uvas de mesa": {
        "url": 'http://vitibrasil.cnpuv.embrapa.br/download/ProcessaMesa.csv'
    }
}

In [104]:
def processa_uvas_processadas(url, tipo_uva):
    cd_tipo_uva_map = {
        "ti": "Tintas",
        "br": "Brancas e Rosadas",
    }
    id_column_name = "ID_" + tipo_uva.replace(" ", "_").upper()
    processamento = (
        pd.read_csv(
            url,
            sep='\t'
        )
        .melt(
            id_vars=["id", "control", "cultivar"],
            var_name="ano",
            value_name="uvas_processadas_viniferas_Kg"
        )
        .rename(columns={
            "id": id_column_name,
            "control": "ID_CONTROLE",
            "cultivar": "NM_CULTIVAR",
            "ano": "DT_ANO",
            "uvas_processadas_viniferas_Kg": "QT_UVAS_PROCESSADAS_VINIFERAS_KG"
        })
        .query("ID_CONTROLE not in ['TINTAS', 'BRANCASEROSADAS']") # titulos com quantidade agregada
    )
    processamento["CD_TIPO_VINHO"] = (
        processamento["ID_CONTROLE"]
        .str.split("_").str[0]
        .map(cd_tipo_uva_map)
    )

    # tratando valores nao numericos na coluna VR_UVAS_PROCESSADAS_VINIFERAS_KG
    valores_unicos = processamento["QT_UVAS_PROCESSADAS_VINIFERAS_KG"].unique()
    valores_unicos_nao_numericos = [x for x in valores_unicos if not isinstance(x, (int, float))]

    pd.set_option('future.no_silent_downcasting', True) # para evitar futuros warnings no replace
    # garantindo os tipos
    processamento = (
        processamento
        .replace('nd', 0)
        .replace('*', np.nan)
        .assign(CD_TIPO_UVA=tipo_uva)
        .astype({
            "ID_UVA_PROCESSADA": int,
            "ID_CONTROLE": str,
            "NM_CULTIVAR": str,
            "DT_ANO": int,
            "CD_TIPO_VINHO": str,
            "CD_TIPO_UVA": str,
            "QT_UVAS_PROCESSADAS_VINIFERAS_KG": float
        })
    )
    assert (processamento.groupby(["ID_UVA_PROCESSADA"])["NM_CULTIVAR"].nunique() == 1).all, "garantir que o nome da uva é única por ID_UVA_PROCESSADA"
    
    return processamento

In [105]:
processamento = pd.concat(
    [processa_uvas_processadas(v["url"], tipo_uva) for tipo_uva, v in processing_dict.items()]
)

In [106]:
processamento.head()

,ID_UVA_PROCESSADA,ID_CONTROLE,NM_CULTIVAR,DT_ANO,QT_UVAS_PROCESSADAS_VINIFERAS_KG,CD_TIPO_VINHO,CD_TIPO_UVA
1,2,ti_Alicante Bouschet,Alicante Bouschet,1970,0.0,Tintas,Viniferas
2,3,ti_Ancelota,Ancelota,1970,0.0,Tintas,Viniferas
3,4,ti_Aramon,Aramon,1970,0.0,Tintas,Viniferas
4,5,ti_Alfrocheiro,Alfrocheiro,1970,0.0,Tintas,Viniferas
5,6,ti_Arinarnoa,Arinarnoa,1970,0.0,Tintas,Viniferas


In [110]:
"Uvas de mesa".replace(" ", "_").upper()

'UVAS_DE_MESA'

In [101]:
processamento.query("CD_TIPO_UVA != 'Viniferas'").head(1)

,ID_UVA_PROCESSADA,ID_CONTROLE,NM_CULTIVAR,DT_ANO,QT_UVAS_PROCESSADAS_VINIFERAS_KG,CD_TIPO_VINHO,CD_TIPO_UVA
1,2,ti_Alicante Bouschet,Alicante Bouschet,1970,0.0,Tintas,Americanas


In [113]:
processamento.groupby(["ID_UVA_PROCESSADA", "CD_TIPO_UVA"])["NM_CULTIVAR"].nunique().sort_values(ascending=False)

ID_UVA_PROCESSADA  CD_TIPO_UVA
29                 Americanas     2
61                 Viniferas      1
63                 Viniferas      1
64                 Viniferas      1
65                 Viniferas      1
                                 ..
32                 Viniferas      1
33                 Americanas     1
                   Viniferas      1
34                 Americanas     1
134                Viniferas      1
Name: NM_CULTIVAR, Length: 198, dtype: int64

In [115]:
processamento.query("ID_UVA_PROCESSADA == 29").groupby(["ID_UVA_PROCESSADA", "NM_CULTIVAR", "CD_TIPO_UVA"]).head(1)

,ID_UVA_PROCESSADA,ID_CONTROLE,NM_CULTIVAR,DT_ANO,QT_UVAS_PROCESSADAS_VINIFERAS_KG,CD_TIPO_VINHO,CD_TIPO_UVA
28,29,ti_Gamay St Romain,Gamay St Romain,1970,0.0,Tintas,Viniferas
10,29,ti_BRS Rúbea,BRS Rúbea,1970,0.0,Tintas,Americanas
28,29,ti_Othello,Othello,1970,33013.0,Tintas,Americanas


In [76]:
(
    processamento
    .groupby(["NM_CULTIVAR", "CD_TIPO_UVA"])
    ["QT_UVAS_PROCESSADAS_VINIFERAS_KG"]
    .sum()
    .reset_index()
    .sort_values(by=["QT_UVAS_PROCESSADAS_VINIFERAS_KG"], ascending=False)
).head(10)

,NM_CULTIVAR,CD_TIPO_UVA,QT_UVAS_PROCESSADAS_VINIFERAS_KG
70,Moscato Branco,Brancas e Rosadas,641830181.0
121,Trebbiano,Brancas e Rosadas,444306945.0
63,Merlot,Tintas,274422818.0
99,Riesling Italico,Brancas e Rosadas,233201919.0
16,Cabernet Sauvignon,Tintas,229412912.0
15,Cabernet Franc,Tintas,227836723.0
12,Barbera,Tintas,139660107.0
23,Chardonnay,Brancas e Rosadas,127656102.0
108,Semillon,Brancas e Rosadas,115452921.0
111,Tannat,Tintas,80952006.0


In [67]:
processamento["ID_CONTROLE"].str.split("_").str[0].value_counts()

ID_CONTROLE
ti    3604
br    3392
Name: count, dtype: int64

In [65]:
(
    processamento
    #.query("ID_CONTROLE.str.startswith('br')")
    .groupby(["ID_CONTROLE", "NM_CULTIVAR"])
    ["VR_UVAS_PROCESSADAS_VINIFERAS_KG"]
    .sum()
    .reset_index()
    .sort_values(
        by=["VR_UVAS_PROCESSADAS_VINIFERAS_KG"],
        ascending=False)
    .head(20)
)

,ID_CONTROLE,NM_CULTIVAR,VR_UVAS_PROCESSADAS_VINIFERAS_KG
34,br_Moscato_branco,Moscato Branco,641830181.0
56,br_Trebbiano,Trebbiano,444306945.0
101,ti_Merlot,Merlot,274422818.0
47,br_Riesling Italico,Riesling Italico,233201919.0
74,ti_Cabernet Sauvignon,Cabernet Sauvignon,229412912.0
73,ti_Cabernet Franc,Cabernet Franc,227836723.0
71,ti_Barbera,Barbera,139660107.0
7,br_Chardonnay,Chardonnay,127656102.0
53,br_Semillon,Semillon,115452921.0
120,ti_Tannat,Tannat,80952006.0


In [55]:
processamento.query("ID_CONTROLE == NM_CULTIVAR")

,ID_PRODUTO,ID_CONTROLE,NM_CULTIVAR,DT_ANO,VR_UVAS_PROCESSADAS_VINIFERAS_KG
0,1,TINTAS,TINTAS,1970,10448228.0
134,1,TINTAS,TINTAS,1971,11012833.0
268,1,TINTAS,TINTAS,1972,10798824.0
402,1,TINTAS,TINTAS,1973,8213674.0
536,1,TINTAS,TINTAS,1974,17457849.0
670,1,TINTAS,TINTAS,1975,22593885.0
804,1,TINTAS,TINTAS,1976,20265190.0
938,1,TINTAS,TINTAS,1977,24830345.0
1072,1,TINTAS,TINTAS,1978,21984546.0
1206,1,TINTAS,TINTAS,1979,25805939.0


In [39]:
processamento.query("DT_ANO != 2022").tail(10)

,ID_PRODUTO,ID_CONTROLE,NM_CULTIVAR,DT_ANO,VR_UVAS_PROCESSADAS_VINIFERAS_KG
7092,125,br_Tocai Friulano,Tocai Friulano,2022,*
7093,126,br_Trebbiano,Trebbiano,2022,*
7094,127,br_Trebbiano Toscano,Trebbiano Toscano,2022,*
7095,128,br_Verdea,Verdea,2022,*
7096,129,br_Verdelho,Verdelho,2022,*
7097,130,br_Verdiso,Verdiso,2022,*
7098,131,br_Vermentino,Vermentino,2022,*
7099,132,br_Vernaccia,Vernaccia,2022,*
7100,133,br_Viognier,Viognier,2022,*
7101,134,br_Outras(3),Outras(3),2022,*


In [48]:
valores_unicos = processamento["VR_UVAS_PROCESSADAS_VINIFERAS_KG"].unique()

valores_unicos_nao_numericos = [x for x in valores_unicos if not isinstance(x, (int, float))]


In [49]:
valores_unicos_nao_numericos

['nd', '*']

In [37]:
(processamento.groupby(["NM_CULTIVAR", "DT_ANO"])["ID_CONTROLE"].nunique() != 1).sum()

0